In [6]:
from __future__ import division
import numpy as np
import pandas as pd
import sklearn as sk
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
import os
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'svg'
from pylab import rcParams
rcParams['figure.figsize']=8,5

In [7]:
#import data
os.chdir("/home/gladkov2/storage/al_R/3new")
os.listdir()
otus = pd.read_csv("otu_table_for_rf.txt", sep="\t")
metadata = pd.read_csv("metadata_for_rf.txt", sep="\t")



In [10]:
# same shit that above with merged repeats
metadata.index = metadata["Unnamed: 0"]
metadata = metadata.drop(columns=["Unnamed: 0"])

otus.index = otus["Unnamed: 0"]
otus = otus.drop(columns=["Unnamed: 0"])

metadata = metadata.drop(columns=["Filename","Description","Plant","Inoculation","Al", "QRTPCR", "SeedCa"])

In [11]:
# Use numpy to convert to arrays
import numpy as np# Labels are the values we want to predict

feature_list = list(metadata)# Convert to numpy array
features = np.array(metadata)
labels = np.array(otus)

In [12]:

# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25, random_state = 42)

In [13]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (46, 104)
Training Labels Shape: (46, 9971)
Testing Features Shape: (16, 104)
Testing Labels Shape: (16, 9971)


In [85]:
metadata

,BineM,SeedM,BineSeedM,SeedN,SeedMOne,NodPerPlant,NodPerPlant.1,Mic,Ves,Arb,...,BineSeedPerPlantCu,BineSeedPerPlantFe,BineSeedPerPlantK,BineSeedPerPlantMg,BineSeedPerPlantMo,BineSeedPerPlantNi,BineSeedPerPlantP,BineSeedPerPlantS,BineSeedPerPlantS.1,BineSeedPerPlantZn
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
belimov.10,5.405,4.735,10.140,39,121.410256,54,21.00,24.300,10.000,4.00,...,253.594300,1137.951584,688.988513,75.231851,91.338888,164.064022,107.842650,230.591683,54.604450,323.808286
belimov.11,5.205,4.070,9.275,38,107.105263,85,20.70,11.300,9.300,7.30,...,229.916932,1235.844731,625.759416,70.574031,92.003684,174.414639,102.019566,206.885163,50.880056,280.945551
belimov.12,6.450,5.470,11.920,36,151.944444,100,30.00,60.700,28.700,36.00,...,219.799087,902.352760,823.867394,94.108222,90.517679,207.040612,128.739554,202.086281,68.266283,311.800693
belimov.13,6.480,4.280,10.760,31,138.064516,107,25.00,2.000,14.700,4.00,...,214.902651,882.726277,1036.016748,85.204781,102.978868,432.093049,31.304417,204.789880,69.358599,193.273730
belimov.14,6.280,5.025,11.305,38,132.236842,118,36.23,2.300,6.000,3.00,...,184.640394,938.947281,1024.193155,85.113614,103.493880,435.846154,34.006418,170.618719,69.992871,253.253837
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
belimov.64,2.580,4.655,7.235,20,232.750000,14,9.00,68.000,30.000,48.00,...,336.588803,1193.748559,471.095851,43.379983,117.645923,265.732726,36.944330,361.888185,42.654965,334.724218
belimov.6,3.585,3.750,7.335,32,117.187500,7,3.00,4.075,6.675,4.85,...,155.655556,763.414103,487.441453,44.048718,68.062821,236.818376,17.406838,138.683120,32.462577,141.307436
belimov.7,4.335,4.110,8.445,36,114.166667,20,5.80,2.000,3.000,2.00,...,175.749375,738.616571,771.147271,56.048829,82.553877,294.869476,20.107772,165.107696,41.644410,162.783097


In [14]:
from sklearn.model_selection import RandomizedSearchCV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = 30)# Fit the random search model
rf_random.fit(train_features, train_labels)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=30)]: Using backend LokyBackend with 30 concurrent workers.
[Parallel(n_jobs=30)]: Done 102 tasks      | elapsed:  5.7min
[Parallel(n_jobs=30)]: Done 300 out of 300 | elapsed: 25.3min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   ccp_alpha=0.0,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   max_samples=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators=100,
                              

In [15]:
rf_random.best_params_`

{'n_estimators': 800,
 'min_samples_split': 10,
 'min_samples_leaf': 4,
 'max_features': 'sqrt',
 'max_depth': 100,
 'bootstrap': True}

In [20]:
rf = RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=100, max_features='sqrt', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=4,
                      min_samples_split=10, min_weight_fraction_leaf=0.0,
                      n_estimators=800, n_jobs=30, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)
rf.fit(train_features, train_labels)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=100, max_features='sqrt', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=4,
                      min_samples_split=10, min_weight_fraction_leaf=0.0,
                      n_estimators=800, n_jobs=30, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [21]:
# Get numerical feature importances
importances = list(rf.feature_importances_)# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 3)) for feature, importance in zip(feature_list, importances)]# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: BineMn               Importance: 0.045
Variable: SeedMg               Importance: 0.037
Variable: pH                   Importance: 0.031
Variable: BineCu               Importance: 0.027
Variable: BineMg               Importance: 0.026
Variable: BineP                Importance: 0.026
Variable: SeedS                Importance: 0.022
Variable: BineM                Importance: 0.02
Variable: SeedN                Importance: 0.02
Variable: BineCo               Importance: 0.02
Variable: BineB                Importance: 0.019
Variable: BineZn               Importance: 0.019
Variable: BinePerPlantMn       Importance: 0.019
Variable: SeedK                Importance: 0.018
Variable: SeedMn               Importance: 0.018
Variable: SeedMo               Importance: 0.018
Variable: BinePerPlantMo       Importance: 0.018
Variable: SeedCo               Importance: 0.017
Variable: SoilMn               Importance: 0.017
Variable: BineAl               Importance: 0.016
Variable: SeedP        

In [22]:
rf.

AttributeError: 'RandomForestRegressor' object has no attribute 'feature_names_'

In [27]:
# Extract single tree
estimator = rf.estimators_[5]
estimator

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=100,
                      max_features='sqrt', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=4, min_samples_split=10,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=611856445, splitter='best')

In [40]:
# Export as dot file
export_graphviz(estimator, out_file = "tree.dot", 
                feature_names=metadata.columns,
                filled=True,
                rounded=True)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 20000, num = 100)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt', '0.5' , '0.2']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 20, 40, 60, 80]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4, 10, 20, 40, 60, 80]
# Method of selecting samples for training each tree
bootstrap = [True, False]# Create the random grid
oob_score = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap,
               'oob_score':oob_score}
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf_1000 = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random_1000 = RandomizedSearchCV(estimator = rf_1000, param_distributions = random_grid, n_iter = 1000, cv = 3, verbose=2, random_state=42, n_jobs = 30)# Fit the random search model
rf_random_1000.fit(train_features, train_labels)
rf_random.best_params_
rf_random_1000.best_params_

Fitting 3 folds for each of 1000 candidates, totalling 3000 fits


[Parallel(n_jobs=30)]: Using backend LokyBackend with 30 concurrent workers.
[Parallel(n_jobs=30)]: Done 102 tasks      | elapsed:  1.7min
[Parallel(n_jobs=30)]: Done 305 tasks      | elapsed:  5.7min


In [46]:
rf_random_1000.best_params_

{'oob_score': True,
 'n_estimators': 5400,
 'min_samples_split': 2,
 'min_samples_leaf': 4,
 'max_features': 'sqrt',
 'max_depth': 110,
 'bootstrap': True}

In [47]:
rf_random.best_params_

{'n_estimators': 800,
 'min_samples_split': 10,
 'min_samples_leaf': 4,
 'max_features': 'sqrt',
 'max_depth': 100,
 'bootstrap': True}

In [48]:
rf_2 = RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=110, max_features='sqrt', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=4,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=5400, n_jobs=30, oob_score=True,
                      random_state=None, verbose=0, warm_start=False)
rf_2.fit(train_features, train_labels)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=110, max_features='sqrt', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=4,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=5400, n_jobs=30, oob_score=True,
                      random_state=None, verbose=0, warm_start=False)

In [49]:
# Get numerical feature importances
importances = list(rf_2.feature_importances_)# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 3)) for feature, importance in zip(feature_list, importances)]# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: BineMn               Importance: 0.038
Variable: SeedMg               Importance: 0.037
Variable: pH                   Importance: 0.028
Variable: BineMg               Importance: 0.027
Variable: BineCo               Importance: 0.022
Variable: BineCu               Importance: 0.02
Variable: BineP                Importance: 0.02
Variable: SeedMo               Importance: 0.02
Variable: SeedS                Importance: 0.02
Variable: BinePerPlantMo       Importance: 0.019
Variable: SeedN                Importance: 0.017
Variable: BineMo               Importance: 0.017
Variable: BineZn               Importance: 0.017
Variable: BinePerPlantMn       Importance: 0.017
Variable: BineM                Importance: 0.016
Variable: BineNi               Importance: 0.016
Variable: NitBine              Importance: 0.015
Variable: SeedCo               Importance: 0.015
Variable: SeedP                Importance: 0.015
Variable: BineB                Importance: 0.014
Variable: BineS         

In [54]:
# Export as dot file
estimator = rf_2.estimators_[5]
export_graphviz(estimator, out_file = "tree2.dot", 
                feature_names=metadata.columns,
                filled=True,
                rounded=True)

In [70]:
estimator.feature_importances_
metadata.columns
imp = list(estimator.feature_importances_)
[print(i) for i in zip(metadata.columns, imp)]
sum(imp)

('BineM', 0.0)
('SeedM', 0.0)
('BineSeedM', 0.0)
('SeedN', 0.26357285318157436)
('SeedMOne', 0.0)
('NodPerPlant', 0.0)
('NodPerPlant.1', 0.0)
('Mic', 0.0)
('Ves', 0.0)
('Arb', 0.0)
('pH', 0.0)
('NitBine', 0.0)
('Nit15', 0.0)
('NitPerPlant', 0.14466566059311806)
('Nit15PerPlant', 0.0)
('NitSeed', 0.0)
('NitSeed15', 0.0)
('NitSeedPerPlant', 0.0)
('Nit15SeedPerPlant', 0.0)
('NitBineSeedPerPlant', 0.0)
('Nit15BineSeedPerPlant', 0.0)
('BineAl', 0.0)
('BineB', 0.0)
('BineCa', 0.0)
('BineCo', 0.15321587833472297)
('BineCu', 0.0)
('BineFe', 0.0)
('BineK', 0.0)
('BineMg', 0.0)
('BineMn', 0.0)
('BineMo', 0.0)
('BineNi', 0.0)
('BineP', 0.27497801003331007)
('BineS', 0.0)
('BineZn', 0.0)
('SeedAl', 0.0)
('SeedB', 0.0)
('SeedCo', 0.0)
('SeedCu', 0.0)
('SeedFe', 0.0)
('SeedK', 0.0)
('SeedMg', 0.0)
('SeedMn', 0.0)
('SeedMo', 0.0)
('SeedNi', 0.0)
('SeedP', 0.0)
('SeedS', 0.0)
('SeedZn', 0.0)
('SoilAl', 0.0)
('SoilB', 0.0)
('SoilCa', 0.0)
('SoilCo', 0.0)
('SoilCu', 0.0)
('SoilFe', 0.0)
('SoilK', 0.0)
(

1.0

In [73]:
from sklearn.model_selection import RandomizedSearchCV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 20000, num = 100)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt', '0.5' , '0.2']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 20, 40, 60, 80]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4, 10, 20, 40, 60, 80]
# Method of selecting samples for training each tree
bootstrap = [True, False]# Create the random grid
oob_score = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap,
               'oob_score':oob_score}
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf_1000_2 = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random_1000_2 = RandomizedSearchCV(estimator = rf_1000_2, param_distributions = random_grid, n_iter = 1000, cv = 3, verbose=2, random_state=42, n_jobs = 30)# Fit the random search model
rf_random_1000_2.fit(train_features, train_labels)



Fitting 3 folds for each of 1000 candidates, totalling 3000 fits


[Parallel(n_jobs=30)]: Using backend LokyBackend with 30 concurrent workers.
[Parallel(n_jobs=30)]: Done 103 tasks      | elapsed:  1.2min
[Parallel(n_jobs=30)]: Done 306 tasks      | elapsed:  4.2min
[Parallel(n_jobs=30)]: Done 589 tasks      | elapsed: 11.6min
[Parallel(n_jobs=30)]: Done 954 tasks      | elapsed: 21.7min
[Parallel(n_jobs=30)]: Done 1399 tasks      | elapsed: 30.6min
[Parallel(n_jobs=30)]: Done 1926 tasks      | elapsed: 45.9min
[Parallel(n_jobs=30)]: Done 2533 tasks      | elapsed: 76.4min
[Parallel(n_jobs=30)]: Done 3000 out of 3000 | elapsed: 165.1min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   ccp_alpha=0.0,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   max_samples=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators=100,
                              

In [75]:
rf_random_1000_2.best_params_


{'oob_score': True,
 'n_estimators': 5400,
 'min_samples_split': 2,
 'min_samples_leaf': 4,
 'max_features': 'sqrt',
 'max_depth': 110,
 'bootstrap': True}

In [76]:
rf_random_1000.best_params_

{'oob_score': True,
 'n_estimators': 5400,
 'min_samples_split': 2,
 'min_samples_leaf': 4,
 'max_features': 'sqrt',
 'max_depth': 110,
 'bootstrap': True}

In [91]:
rf_2.estimators_[1]

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=110,
                      max_features='sqrt', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=4, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=902224153, splitter='best')

In [92]:
metadata

,BineM,SeedM,BineSeedM,SeedN,SeedMOne,NodPerPlant,NodPerPlant.1,Mic,Ves,Arb,...,BineSeedPerPlantCu,BineSeedPerPlantFe,BineSeedPerPlantK,BineSeedPerPlantMg,BineSeedPerPlantMo,BineSeedPerPlantNi,BineSeedPerPlantP,BineSeedPerPlantS,BineSeedPerPlantS.1,BineSeedPerPlantZn
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
belimov.10,5.405,4.735,10.140,39,121.410256,54,21.00,24.300,10.000,4.00,...,253.594300,1137.951584,688.988513,75.231851,91.338888,164.064022,107.842650,230.591683,54.604450,323.808286
belimov.11,5.205,4.070,9.275,38,107.105263,85,20.70,11.300,9.300,7.30,...,229.916932,1235.844731,625.759416,70.574031,92.003684,174.414639,102.019566,206.885163,50.880056,280.945551
belimov.12,6.450,5.470,11.920,36,151.944444,100,30.00,60.700,28.700,36.00,...,219.799087,902.352760,823.867394,94.108222,90.517679,207.040612,128.739554,202.086281,68.266283,311.800693
belimov.13,6.480,4.280,10.760,31,138.064516,107,25.00,2.000,14.700,4.00,...,214.902651,882.726277,1036.016748,85.204781,102.978868,432.093049,31.304417,204.789880,69.358599,193.273730
belimov.14,6.280,5.025,11.305,38,132.236842,118,36.23,2.300,6.000,3.00,...,184.640394,938.947281,1024.193155,85.113614,103.493880,435.846154,34.006418,170.618719,69.992871,253.253837
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
belimov.64,2.580,4.655,7.235,20,232.750000,14,9.00,68.000,30.000,48.00,...,336.588803,1193.748559,471.095851,43.379983,117.645923,265.732726,36.944330,361.888185,42.654965,334.724218
belimov.6,3.585,3.750,7.335,32,117.187500,7,3.00,4.075,6.675,4.85,...,155.655556,763.414103,487.441453,44.048718,68.062821,236.818376,17.406838,138.683120,32.462577,141.307436
belimov.7,4.335,4.110,8.445,36,114.166667,20,5.80,2.000,3.000,2.00,...,175.749375,738.616571,771.147271,56.048829,82.553877,294.869476,20.107772,165.107696,41.644410,162.783097
